In [16]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

def load_txt(filename):
    with open(filename,'r') as df:
        lines = df.readlines()
        dataset = []
        for line in lines:
            tem = line.split()
            x = [float(i) for i in tem]
            dataset.append(x)
    return dataset

def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] != predicted[i]:
            correct += 1
    return correct / float(len(actual)) 

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(train_set ,test_set ,algorithm, *args):
    
    predicted = algorithm(train_set, test_set, *args)
    actual = [row[-1] for row in test_set]
    accuracy = accuracy_metric(actual, predicted)

    return accuracy

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    #print(max(set(predictions), key=predictions.count))
    return max(set(predictions), key=predictions.count)

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        print('Tree%s'%i)
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

# Test the random forest algorithm
seed(2)
# load and prepare data
dataset = load_txt('train.txt')
test_set = load_txt('test.txt')
# evaluate algorithm
max_depth = 20
min_size = 1
sample_size = 0.5
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [100]:
    scores = evaluate_algorithm(dataset, test_set ,random_forest, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Q15: %s' % scores)

Tree0
Tree1
Tree2
Tree3
Tree4
Tree5
Tree6
Tree7
Tree8
Tree9
Tree10
Tree11
Tree12
Tree13
Tree14
Tree15
Tree16
Tree17
Tree18
Tree19
Tree20
Tree21
Tree22
Tree23
Tree24
Tree25
Tree26
Tree27
Tree28
Tree29
Tree30
Tree31
Tree32
Tree33
Tree34
Tree35
Tree36
Tree37
Tree38
Tree39
Tree40
Tree41
Tree42
Tree43
Tree44
Tree45
Tree46
Tree47
Tree48
Tree49
Tree50
Tree51
Tree52
Tree53
Tree54
Tree55
Tree56
Tree57
Tree58
Tree59
Tree60
Tree61
Tree62
Tree63
Tree64
Tree65
Tree66
Tree67
Tree68
Tree69
Tree70
Tree71
Tree72
Tree73
Tree74
Tree75
Tree76
Tree77
Tree78
Tree79
Tree80
Tree81
Tree82
Tree83
Tree84
Tree85
Tree86
Tree87
Tree88
Tree89
Tree90
Tree91
Tree92
Tree93
Tree94
Tree95
Tree96
Tree97
Tree98
Tree99
Trees: 100
Q15: 0.164


In [15]:
max_depth = 20
min_size = 1
sample_size = 0.5
n_features = int(sqrt(len(dataset[0])-1))
eout = 0
for n_trees in range(1,2001):
    scores = evaluate(dataset, test_set ,random_forest, max_depth, min_size, 
                                sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    eout += scores
print('Q15: %s'%eout/2000)

Tree0
Tree1
Tree2
Tree3
Tree4
Tree5
Tree6
Tree7
Tree8
Tree9
Tree10
Tree11
Tree12
Tree13
Tree14
Tree15
Tree16
Tree17
Tree18
Tree19
Tree20
Tree21
Tree22
Tree23
Tree24
Tree25
Tree26
Tree27
Tree28
Tree29
Tree30
Tree31
Tree32
Tree33
Tree34
Tree35
Tree36
Tree37
Tree38
Tree39
Tree40
Tree41
Tree42
Tree43
Tree44
Tree45
Tree46
Tree47
Tree48
Tree49
Tree50
Tree51
Tree52
Tree53
Tree54
Tree55
Tree56
Tree57
Tree58
Tree59
Tree60
Tree61
Tree62
Tree63
Tree64
Tree65
Tree66
Tree67
Tree68
Tree69
Tree70
Tree71
Tree72
Tree73
Tree74
Tree75
Tree76
Tree77
Tree78
Tree79
Tree80
Tree81
Tree82
Tree83
Tree84
Tree85
Tree86
Tree87
Tree88
Tree89
Tree90
Tree91
Tree92
Tree93
Tree94
Tree95
Tree96
Tree97
Tree98
Tree99
Tree100
Tree101
Tree102
Tree103
Tree104
Tree105
Tree106
Tree107
Tree108
Tree109
Tree110
Tree111
Tree112
Tree113
Tree114
Tree115
Tree116
Tree117
Tree118
Tree119
Tree120
Tree121
Tree122
Tree123
Tree124
Tree125
Tree126
Tree127
Tree128
Tree129
Tree130
Tree131
Tree132
Tree133
Tree134
Tree135
Tree136
Tree137
Tree13

Tree1034
Tree1035
Tree1036
Tree1037
Tree1038
Tree1039
Tree1040
Tree1041
Tree1042
Tree1043
Tree1044
Tree1045
Tree1046
Tree1047
Tree1048
Tree1049
Tree1050
Tree1051
Tree1052
Tree1053
Tree1054
Tree1055
Tree1056
Tree1057
Tree1058
Tree1059
Tree1060
Tree1061
Tree1062
Tree1063
Tree1064
Tree1065
Tree1066
Tree1067
Tree1068
Tree1069
Tree1070
Tree1071
Tree1072
Tree1073
Tree1074
Tree1075
Tree1076
Tree1077
Tree1078
Tree1079
Tree1080
Tree1081
Tree1082
Tree1083
Tree1084
Tree1085
Tree1086
Tree1087
Tree1088
Tree1089
Tree1090
Tree1091
Tree1092
Tree1093
Tree1094
Tree1095
Tree1096
Tree1097
Tree1098
Tree1099
Tree1100
Tree1101
Tree1102
Tree1103
Tree1104
Tree1105
Tree1106
Tree1107
Tree1108
Tree1109
Tree1110
Tree1111
Tree1112
Tree1113
Tree1114
Tree1115
Tree1116
Tree1117
Tree1118
Tree1119
Tree1120
Tree1121
Tree1122
Tree1123
Tree1124
Tree1125
Tree1126
Tree1127
Tree1128
Tree1129
Tree1130
Tree1131
Tree1132
Tree1133
Tree1134
Tree1135
Tree1136
Tree1137
Tree1138
Tree1139
Tree1140
Tree1141
Tree1142
Tree1143
Tree1144
T

Tree1945
Tree1946
Tree1947
Tree1948
Tree1949
Tree1950
Tree1951
Tree1952
Tree1953
Tree1954
Tree1955
Tree1956
Tree1957
Tree1958
Tree1959
Tree1960
Tree1961
Tree1962
Tree1963
Tree1964
Tree1965
Tree1966
Tree1967
Tree1968
Tree1969
Tree1970
Tree1971
Tree1972
Tree1973
Tree1974
Tree1975
Tree1976
Tree1977
Tree1978
Tree1979
Tree1980
Tree1981
Tree1982
Tree1983
Tree1984
Tree1985
Tree1986
Tree1987
Tree1988
Tree1989
Tree1990
Tree1991
Tree1992
Tree1993
Tree1994
Tree1995
Tree1996
Tree1997
Tree1998
Tree1999
Trees: 2000
Q15: 0.163


In [17]:
import numpy as np

In [30]:
x = [1 for i in range(40)]
y = [0 for i in range(60)]

In [29]:
for _ in range(100):
    g1 = np.random.permutation(x+y)
    g2 = np.random.permutation(x+y)
    g3 = np.random.permutation(x+y)
    g4 = np.random.permutation(x+y)
    g5 = np.random.permutation(x+y)

n = 0
for i in range(100):
    if (g1[i]+g2[i]+g3[i]+g4[i]+g5[i]>=3):
        n += 1
n / 100

0.33

In [25]:
x = np.array([0,0])
x = x.T
y = [1,1]
y = np.array(y).T
np.hstack((x,y))

array([0, 0, 1, 1])